In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

In [3]:

# Função para carregar o arquivo CSV
def load_data(file_path):
    try:
        # Lê o arquivo a partir da terceira linha (pulando as duas primeiras)
        df = pd.read_csv(
            file_path,
            delimiter=';',
            decimal=',',
            skipinitialspace=True,
            encoding='ISO-8859-1',
            skiprows=1  # Ignorar as 2 primeiras linhas
        )

        # Verificar se o número de colunas bate com a renomeação
        if len(df.columns) == 4:  # Substitua pelo número esperado de colunas
            df.columns = [
                'Tempo (s)', 'Deslocamento (mm)', 'Força (kN)',
                'Tensão à flexão (MPa)'
            ]
        else:
            print("Número de colunas no CSV não corresponde ao esperado.")
            print(f"Colunas encontradas: {df.columns}")
            return None

        # Converte apenas as colunas numéricas para float
        colunas_numericas = [
            'Tempo (s)', 'Deslocamento (mm)', 'Força (kN)',
            'Tensão à flexão (MPa)'
        ]

        for col in colunas_numericas:
            if col in df.columns:  # Verifica se a coluna existe antes de processar
                # Verifica se a coluna é do tipo 'object' (string) antes de usar .str.replace
                if df[col].dtype == 'object':
                    df[col] = df[col].str.replace(',', '.', regex=False)
                # Converte para numérico
                df[col] = pd.to_numeric(df[col], errors='coerce')

        return df

    except Exception as e:
        print(f"Erro ao carregar o arquivo: {e}")
        return None

# Caminho para o arquivo CSV
file_path = r'C:/Users/joaob/OneDrive/Área de Trabalho/Relatorios MecSol/Ensaios Lab 2/Dados Nylon/NYLON 70_2_2.csv'
# Carregar os dados
df = load_data(file_path)

# Ver os primeiros registros para conferir
if df is not None:
    print(df.head())
else:
    print("Erro ao carregar os dados.")


Erro ao carregar o arquivo: [Errno 2] No such file or directory: 'C:/Users/joaob/OneDrive/Área de Trabalho/Relatorios MecSol/Ensaios Lab 2/Dados Nylon/NYLON 70_2_2.csv'
Erro ao carregar os dados.


In [3]:
df['Deformação (%)'] = (df['Deslocamento (mm)'] / 101.15) * 100

TypeError: 'NoneType' object is not subscriptable

In [4]:
df

In [ ]:
from scipy import stats

def calc_modulo_elasticidade_dinamico(df, limite_erro=0.01): #como entrada um DataFrame df e um limite_erro, que determina quão próximo da linearidade os dados devem estar.
    # Filtra valores de tensão positivos
    filtro = df[df['Tensão à flexão (MPa)'] > 0]

    # Inicializa variáveis
    n_max = len(filtro)
    modulo_elasticidade = np.nan

    for n in range(2, n_max):  # Começar com 2 pontos
        tensao = filtro['Tensão à flexão (MPa)'].iloc[:n].values      #Para cada iteração do loop, os dados de tensão e deformação são extraídos até o índice n. A deformação é convertida de porcentagem para fração dividindo por 100.
        deformacao = filtro['Deformação (%)'].iloc[:n].values / 100

        # Verifica se todos os valores de deformação ou tensão são iguais
        if np.all(deformacao == deformacao[0]) or np.all(tensao == tensao[0]):
            continue  # Ignora este conjunto e passa para o próximo

        # Realiza a regressão linear
        slope, intercept, r_value, p_value, std_err = stats.linregress(deformacao, tensao) #Realiza a regressão linear nos dados de tensão e deformação.
                                                                                           #slope representa o coeficiente angular da linha de melhor ajuste,
                                                                                           # que é o que queremos calcular como o módulo de elasticidade.

        # Prediz a tensão usando a linha de melhor ajuste
        tensao_predita = slope * deformacao + intercept

        # Prediz os valores de tensão com base na linha de melhor ajuste e calcula o Erro Quadrático Médio (EQM) entre os valores reais e os preditos.
        eqm = np.mean((tensao - tensao_predita) ** 2)


#Se o EQM for menor que o limite definido, atualiza modulo_elasticidade com o valor do coeficiente angular (slope).
        # Verifica se o EQM é menor que o limite
        if eqm < limite_erro:
            modulo_elasticidade = slope  # O coeficiente angular é o módulo de elasticidade
        else:
            break  # Sai do loop se a linearidade não se mantiver

    return modulo_elasticidade

# Exemplo de uso
modulo_elasticidade_dinamico = calc_modulo_elasticidade_dinamico(df)

if not np.isnan(modulo_elasticidade_dinamico):
    print(f"Módulo de Elasticidade (dinâmico): {modulo_elasticidade_dinamico:.2f} MPa")
else:
    print("Não foi possível calcular o módulo de elasticidade.")

Módulo de Elasticidade (dinâmico): 749.47 MPa


In [5]:

# Cálculo da Resistência Máxima
resistencia_maxima = df['Tensão à flexão (MPa)'].max()

# Cálculo da Deformação na Resistência Máxima
deformacao_resistencia_maxima = df.loc[df['Tensão à flexão (MPa)'].idxmax(), 'Deformação (%)']

# Cálculo da Deformação na Resistência Máxima em mm
deformacao_resistencia_maxima_mm = (deformacao_resistencia_maxima / 100) * 100.68 #Colocar  h

# Cálculo da Resistência na Ruptura
resistencia_na_ruptura = df['Tensão à flexão (MPa)'].iloc[-1]

# Cálculo da Deformação na Ruptura em mm
deformacao_na_ruptura = df['Deslocamento (mm)'].iloc[-1]

# Exibindo os resultados
print(f"Resistência Máxima: {resistencia_maxima} MPa")
print(f"Deformação na Resistência Máxima: {deformacao_resistencia_maxima} %")
print(f"Deformação na Resistência Máxima (mm): {deformacao_resistencia_maxima_mm:.2f} mm")
print(f"Resistência na Ruptura: {resistencia_na_ruptura} MPa")
print(f"Deformação na Ruptura (mm): {deformacao_na_ruptura:.2f} mm")


TypeError: 'NoneType' object is not subscriptable